In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [3]:
raw_data = pd.read_csv("Data/AMAZON_FASHION.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [4]:
data

,User,Item,Rating
408192,A1E2AYRZNY0CWP,B000VOEEGG,5.0
869111,A2QP0K4MR0IAY,B01GG71TA0,4.0
592282,A2GUXG7TZ42KOO,B00RQONTWE,2.0
876776,A1DJ5TOKOZ8HKC,B01H2TR5LS,1.0
538683,A36BXB7XUAWPJY,B00LXWSYQG,5.0
...,...,...,...
119116,A5KTZ9A4OGHJI,B00CWEZZQ6,5.0
847551,A31449U600RDUY,B01EZQO83Y,4.0
365629,A30Z4EOV284I4,B01CQAFN5I,5.0
698277,A2U5BGBT47AC1R,B014KBVK8U,5.0


In [5]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B00007GDFV,B000085HZL,B00008JPRZ,B00008JQLL,B0000A98WH,B0000BUWK5,B0000E62U3,B0000ERQV7,B0001LU08U,B00024R0NO,...,B01HHV6UYE,B01HI0IEGG,B01HI9YG0U,B01HILYS88,B01HIVIUAU,B01HIWLG64,B01HIZ7VJC,B01HJ3SYZS,B01HJEOC9E,B01HJFNU7S
User,,,,,,,,,,,,,,,,,,,,,
A0696532YENECYUDV1DS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0864963DOAY7LXGS5I6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0872885ORBGL7IE404O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1008ZNQFUIVAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100ZDJR3G83NY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0696532YENECYUDV1DS,A0864963DOAY7LXGS5I6,A0872885ORBGL7IE404O,A1008ZNQFUIVAC,A100ZDJR3G83NY,A1011525S8J79H,A1014T4QF49PCM,A101QML4Q2HHMB,A10218UDBWESJJ,A1026G7PUNJGQN,...,AZVUXPIDA9VAO,AZW3PMMOEWAPQ,AZW959D8EMNZ,AZWGAXATL9CJW,AZYIKW3LBIV03,AZYNASCEZ6FXX,AZYSIV9D788V1,AZZQ6TNIU4DN8,AZZXCFBNEWIBQ,AZZXKG9AGRVGU
Item,,,,,,,,,,,,,,,,,,,,,
B00007GDFV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B000085HZL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00008JPRZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00008JQLL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0000A98WH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(7482, 10)

In [8]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(7482, 7482)

In [9]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

4433

In [10]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(7482,)

In [11]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B000A2TG1C',
 'B000AL4RXA',
 'B000BD7SGK',
 'B000JOOR7O',
 'B000RAOQ1M',
 'B0016P81YU',
 'B001CCH8JQ',
 'B001DMM0NE',
 'B001F8LJ92',
 'B001FOOG0U',
 'B001S3EMZ2',
 'B002PIOSRG',
 'B003IS3DWI',
 'B003X4ZTQ0',
 'B004Y9ZNNC',
 'B0057A0KVC',
 'B005PVO9CE',
 'B005UECZ0O',
 'B0076XGFKI',
 'B007F0ZGMK',
 'B007W9ZVME',
 'B0089Q8V8U',
 'B009046YFW',
 'B00928XTRC']

In [28]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_AMAZON_FASHION.json.gz')
data_set = review_data[:30000]
data_set
    

,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,[https://images-na.ssl-images-amazon.com/image...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,[https://images-na.ssl-images-amazon.com/image...,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",[https://images-na.ssl-images-amazon.com/image...,Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"22,005,842inClothing,ShoesJewelry(",5 star,B00F5V4JJ2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"18,933,551inClothing,ShoesJewelry(",5 star5 star (0%),B00F5V4RDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"5,687,062inClothing,ShoesJewelry(",jw00541,B00F5V9758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"24,206,139inClothing,ShoesJewelry(",5 star,B00F5V4IAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [34]:
top25

,ID
0,B000A2TG1C
1,B000AL4RXA
2,B000BD7SGK
3,B000JOOR7O
4,B000RAOQ1M
5,B0016P81YU
6,B001CCH8JQ
7,B001DMM0NE
8,B001F8LJ92
9,B001FOOG0U


In [35]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')

,ID,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,B000A2TG1C,Dolphins Reebok Men's NFL Player Long-Sleeve Mock,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,5 star,B000A2TG1C,[This Reebok NFL Player Long-Sleeve Mock is pe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B000AL4RXA,Bostonian Men's Malden Wingtip Oxford,[https://images-na.ssl-images-amazon.com/image...,Bostonian,"[100% Leather, Imported, Leather sole, Heel me...","998,269inClothing,Shoesamp;Jewelry(",5 star,B000AL4RXA,[Travel the corporate track with style and fle...,$95.78 - $138.00,"[B07BP2YBL3, B00O5YZUCA, B000BW8L0S, B01IDYTEW...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
2,B000BD7SGK,Gem Avenue 925 Sterling Silver Princess Cut Cl...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,0.02 Grams,B000BD7SGK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B000JOOR7O,"Vans Adult Classic Slip-On, (Checkerboard) Bla...",[https://images-na.ssl-images-amazon.com/image...,Vans,[Shipping Information:\n \n...,"42,109inClothing,ShoesJewelry(",5 star,B000JOOR7O,[The shoe that started it all. The iconic Vans...,$48.99 - $150.00,NaN,NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
4,B000RAOQ1M,West Coast Jewelry Grooved Stainless Steel Men...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Product Dimensions:\n \n7 ...,"2,272,685inClothing,ShoesJewelry(",7 inches,B000RAOQ1M,NaN,NaN,"[B00064UU1O, B07MNDJL5V, B00Y8DDDO0, B003G5OFZ...","[B07MNDJL5V, B00064UU1O, B0046ZWLIY, B07CRDSCSS]",NaN,NaN,NaN,NaN
5,B0016P81YU,Chisel 14K Gold Plated Grooved Polished Titani...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,5 star,B0016P81YU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,B001CCH8JQ,California Costumes Women's Sexy Super Model B...,[https://images-na.ssl-images-amazon.com/image...,California Costumes,"[100% Synthetic Fiber, Imported, Hand Wash, Wi...","236,332inClothing,ShoesJewelry(",5 star,B001CCH8JQ,"[Sexy Super Model Adult Wig For that sexy, gla...",$12.47,"[B07GD3L7XQ, B017UA8SX8, B07HH11ZQK, B000VZ5TF...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
7,B001DMM0NE,St. Michael Scapular - Regular Cord,NaN,Rose Scapular,[Package Dimensions:\n \n5....,"4,228,209inClothing,ShoesJewelry(",5 star,B001DMM0NE,NaN,$8.99,NaN,NaN,NaN,NaN,NaN,NaN
8,B001F8LJ92,"Ear Mitts Bandless Ear Muffs For Men & Women, ...",[https://images-na.ssl-images-amazon.com/image...,Ear Mitts,[Slip-on bandless Ear Mitts fit comfortably an...,"14,422inClothing,ShoesJewelry(",5 star,B001F8LJ92,[Ear Mitts are designed to fit securely on you...,$13.95 - $34.95,"[B004RZDC2M, B076VRMRMK, B01LZGPNAI, B01ARKDZ1...","[B004RZDC2M, B076VRMRMK, B01ARKDZ1S, B01LZGPNA...",NaN,NaN,NaN,NaN
9,B001FOOG0U,var aPageStart = (new Date()).getTime();\nvar ...,[https://images-na.ssl-images-amazon.com/image...,Ear Mitts,[Slip-on bandless Ear Mitts fit comfortably an...,"14,704inClothing,ShoesJewelry(",5 star,B001FOOG0U,[Ear Mitts are designed to fit securely on you...,$13.95 - $34.95,"[B004RZDC2M, B076VRMRMK, B01LZGPNAI, B01ARKDZ1...","[B004RZDC2M, B076VRMRMK, B01ARKDZ1S, B01LZGPNA...",NaN,NaN,NaN,NaN


In [36]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [38]:
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[Top25Recommend.columns[:2]]

,ID,title
0,B000A2TG1C,Dolphins Reebok Men's NFL Player Long-Sleeve Mock
1,B000AL4RXA,Bostonian Men's Malden Wingtip Oxford
2,B000BD7SGK,Gem Avenue 925 Sterling Silver Princess Cut Cl...
3,B000JOOR7O,"Vans Adult Classic Slip-On, (Checkerboard) Bla..."
4,B000RAOQ1M,West Coast Jewelry Grooved Stainless Steel Men...
5,B0016P81YU,Chisel 14K Gold Plated Grooved Polished Titani...
6,B001CCH8JQ,California Costumes Women's Sexy Super Model B...
7,B001DMM0NE,St. Michael Scapular - Regular Cord
8,B001F8LJ92,"Ear Mitts Bandless Ear Muffs For Men & Women, ..."
9,B001FOOG0U,var aPageStart = (new Date()).getTime();\nvar ...
